# Лабораторна робота номер 2
### ФБ-24 Довгаль Анна

1. Створити env в якому будуть встановлені всі необхідні бібліотеки та
налаштування для данної лабораторної роботи


```conda
conda create --name adenv python
conda activate adenv
conda install pandas
conda install urllib.request
```

2. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс
(номер) області. При зберіганні файлу до його імені потрібно додати
дату та час завантаження;

In [ ]:
import urllib
import datetime
import os

def get_csv(i):
    if not os.path.exists('csv'):
        os.makedirs('csv')
        
    url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean'.format(i)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    out = open('.\\csv\\NOAA_ID_'+str(i)+'_'+date_and_time+'.csv','wb')
    out.write(text)
    out.close()

for i in range(1,28):
    get_csv(i)

3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу
із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для
сприйняття (не повинно бути спеціалізованих символів, пробілів
тощо). Ця задача має бути реалізована у вигляді окремої процедури,
яка на вхід приймає шлях до директорії, в якій зберігаються файли;


In [1]:
import pandas as pd
import os


def to_df(path):
    dfs=[]
    for file in os.listdir(path):
        if file.endswith('.csv'):
            headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
            df = pd.read_csv(os.path.join(path, file), header = 1, index_col=False, names = headers)
            parts = file.split("_")
            index = int(parts[2])
            df['Area'] = index

            dfs.append(df)

    df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)

    df = df.drop(df.loc[df['VHI'] == -1].index)
    df.replace({"<tt><pre>1982": "1982"}, inplace=True)
    df.drop(df[df['Year'] == '</pre></tt>'].index, inplace=True)
    df['Year'] = pd.to_numeric(df['Year'])
    return df
    


df = to_df('.\\csv\\')
df.head()


,Year,Week,SMN,SMT,VCI,TCI,VHI,Area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,10
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,10
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,10
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,10
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,10


In [2]:
df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Area
60350,2024,5.0,0.101,275.13,58.55,8.77,33.66,9
60351,2024,6.0,0.110,276.43,60.83,9.08,34.96,9
60352,2024,7.0,0.120,277.97,61.35,10.45,35.90,9
60353,2024,8.0,0.130,279.15,60.18,13.87,37.02,9
60354,2024,9.0,0.142,280.64,59.61,16.63,38.12,9


4. Реалізувати процедуру, яка змінить індекси областей, які використані
на порталі NOAA на дані:

In [4]:
def convert_ids(df):
    correct_ids = {1 : 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 26, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 20: 27, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    df['Area'].replace(correct_ids, inplace = True)
    return df


convert_ids(df)
df.head()

C:\Temp\ipykernel_11848\358044620.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Area'].replace(correct_ids, inplace = True)


,Year,Week,SMN,SMT,VCI,TCI,VHI,Area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,21
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,21
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,21
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,21
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,21


5. Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік, пошук екстремумів (min та
max);
- Ряд VHI за вказаний діапазон років для вказаних областей;
виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);
- Аналогічно для помірних посух

In [5]:
def get_extremum(df, area, year):
    min_v = df[(df["Area"] == area) & (df["Year"] == year)]['VHI'].min()
    max_v = df[(df["Area"] == area) & (df["Year"] == year)]['VHI'].max()
    return min_v, max_v
    

selected = df[(df["Area"] == 7) & (df["Year"] == 2005)]['VHI']

min, max = get_extremum(df, 7, 2010)
print('min: ', min, '\nmax: ', max)

min:  25.39 
max:  62.13


In [6]:
filtered = df[(df['Year'] >= 2000) & (df['Year'] <= 2003) & (df['Area'].isin([2, 6, 7]))]
print(filtered[['Year', 'Area', 'VHI']])

       Year  Area    VHI
32254  2000     6  36.69
32255  2000     6  40.87
32256  2000     6  43.27
32257  2000     6  44.39
32258  2000     6  44.24
...     ...   ...    ...
39168  2003     7  41.47
39169  2003     7  44.48
39170  2003     7  49.20
39171  2003     7  54.47
39172  2003     7  59.63

[621 rows x 3 columns]


In [7]:
def ext_droughts(df, pers):
    total = 27  
    years = df[df['VHI'] <= 15].groupby('Year')['Area'].nunique() #рахує кількість областей на кожен рік з посухою
    overpers = years[years > total * (pers / 100)]
    return overpers
    
persnt = 20
print(f"Роки з екстремальними посухами, які торкнулися більше {persnt}% областей по Україні:\n{ext_droughts(df, persnt)}")


Роки з екстремальними посухами, які торкнулися більше 20% областей по Україні:
Year
2000    6
Name: Area, dtype: int64


In [9]:
def mild_droughts(df, pers):
    total = 27  
    years = df[(df['VHI'] <= 35) & (df['VHI'] > 15)].groupby('Year')['Area'].nunique() 
    overpers = years[years > total * (pers / 100)]
    return overpers
    
pers = 90
print(f"Роки з помірними посухами, які торкнулися більше {pers}% областей по Україні:\n{mild_droughts(df, pers)}")


Роки з помірними посухами, які торкнулися більше 90% областей по Україні:
Year
1982    27
1983    26
1984    26
1985    26
1986    27
1989    26
1990    25
1993    25
1995    26
1996    26
2000    27
2019    27
Name: Area, dtype: int64


In [ ]:
df.to_csv("lab2.csv")